### Part 2: Data Acquisition

Below, we filter data from the National Student Survey, as well as from the website Reddit.com. We also import all of the modules we will use throughout the project.

In [105]:
import pandas as pd
import pickle
import json
import requests.auth

### NSS Data

The National Student Survey gathers feedback from students concerning student satisfaction in regards to their course, teaching, and other aspects of university. It is collected from final year undergraduate students across UK universities. Multiple other university comparison websites such as "the Complete University Guide" reference the NSS, a testament to its reliability.

The data is displayed on the NSS website as a series of excel files, with the name of each file being the 'UK Provider Reference Number' of each university. We renamed the files to be used and shifted them to the 'Data' folder of the Report.

For this project we have chosen to compare LSE to the following universities:

- University of Oxford
- University College London
- Birmingham University
- Edinburgh University
- Glasgow University
- Imperial College London
- Kings College London
- Manchester University
- Norwich University of the Arts
- Strathclyde University
- Warwick University

We chose these universities as they encompass a wide range of different categories; some can be closely compared to LSE, such as UCL, KCL, and Imperial, all in London. Other universities are in the Northern parts of the UK, whilst others are specialised in completely different subjects to LSE (Norwich). We believe this will make comparisons more interesting.


We store all of the dataframes (which we clean by removing unnecessary columns and formating correctly)* into a dictionary univ_df.

*For example, since the NSS combines student's responses to the Questions into one overall \"Positivity Measure\", which is the proportion of respondents who gave a positive answer, we only need that, rather than the actual responses to the questions (e.g. Option 1, Option 2, etc.). This does lose some of the núance to the data, but makes visualisation and understanding the data easier.

In [5]:
university_names = [
    "LSE",
    "Oxford",
    "UCL",
    "Birmingham",
    "Edinburgh",
    "Glasgow",
    "Imperial",
    "KCL",
    "Manchester",
    "Norwich",
    "Strathclyde",
    "Warwick"
]

In [6]:
#storing all of the university data in a dictionary DataFrame
univ_df = {}

for university_name in university_names:
    file_path = './data/' + university_name + '_dat.xlsx'
    df = pd.read_excel(file_path, sheet_name="Teaching", header=3)
    
    # Drop unnecessary columns
    columns_to_drop = [
        "Mode of study",
        "Suppression reason",
        "Option 1",
        "Option 2",
        "Option 3",
        "Option 4",
        "Option 5",
        "This does not apply to me",
        "Benchmark (%)",
        "Difference (ppt)",
        "Contribution to benchmark (%)",
        "Materially below benchmark (%)",
        "Materially above benchmark (%)",
        "Publication response headcount",
        "Publication response rate (%)",
        "Broadly in line with benchmark (%)"
    ]
    df_cleaned = df.drop(columns_to_drop, axis=1)
    
    # Store cleaned dataframe in the dictionary
    univ_df[university_name] = df_cleaned


In [7]:
univ_df['Norwich'].head()

,Level of study,Subject level,Subject code,Subject,Question,Responses,Population,Positivity measure (%),Standard deviation
0,All undergraduates,CAH1,CAH11,Computing,Q01: How good are teaching staff at explaining...,92.8,118.5,80.9,3.8
1,All undergraduates,CAH1,CAH11,Computing,Q02: How often do teaching staff make the subj...,92.8,118.5,74.1,4.4
2,All undergraduates,CAH1,CAH11,Computing,Q03: How often is the course intellectually st...,92.8,118.5,73.0,4.2
3,All undergraduates,CAH1,CAH11,Computing,Q04: How often does your course challenge you ...,91.8,118.5,76.6,4.1
4,All undergraduates,CAH1,CAH11,Computing,Q05: To what extent have you had the chance to...,92.8,118.5,81.9,4.1


In [8]:
univ_df['LSE'].head()

,Level of study,Subject level,Subject code,Subject,Question,Responses,Population,Positivity measure (%),Standard deviation
0,All undergraduates,CAH1,CAH04,Psychology,Q01: How good are teaching staff at explaining...,30.0,30.0,100.0,3.7
1,All undergraduates,CAH1,CAH04,Psychology,Q02: How often do teaching staff make the subj...,30.0,30.0,96.7,6.1
2,All undergraduates,CAH1,CAH04,Psychology,Q03: How often is the course intellectually st...,30.0,30.0,100.0,4.7
3,All undergraduates,CAH1,CAH04,Psychology,Q04: How often does your course challenge you ...,30.0,30.0,100.0,5.1
4,All undergraduates,CAH1,CAH04,Psychology,Q05: To what extent have you had the chance to...,30.0,30.0,96.7,5.7


We store the file using the pickle module as it is the easiest to retrieve in our Report Notebook.

In [13]:
with open('./data/univ_df.pkl','wb') as pickle_file:
    pickle.dump(univ_df, pickle_file)

We also want to assess how LSE evolved over the years. The NSS before 2023 takes a different format, where all data is displayed as one large excel file for each year. We took these files straight from the NSS website archives and renamed them to their respective years.

Unfortunately, the format and questions of the NSS changed in 2023 and so comparisons between years before 2023 with 2023 cannot be exact. However, we decided to acknowledge this factor, yet still continue with the analysis. The cleaning and merging of these data will be carried out in the Report notebook.

### Reddit Data

For the Reddit API data, we decided to focus on the subreddit 'UniUK' that has 151 thousand members and is in the top 1% of communities ranked by size. We noticed few posts directly used the phrase 'student satisfaction' when discussing LSE, which we assumed was due to the informal nature of the site. 

Instead, we searched for posts that contained the word 'lse'. We retreived the top thirty posts under this search criteria, and stored the 'Post Title', 'Score' (number of 'upvotes' by other redditers) and 'Top Comment' in a pandas data frame 'df'. This dataframe was then saved to a csv file "Data/reddit_data.csv" so that it can accessed from any other files, and crucially, so that the data only needs to be fetched from Reddit.com once. Since the contents of the subreddit is likely to change frequently, it would be inefficent to perform effective analysis using realtime data, so we instead stored data from the site on 12/04/2024.

We chose to use the website Reddit to acquire data for many reasons. Firstly, their API is easy to use, reliable and openly accessible to the public. This allowed us to effiently access user-generated content without the computational and legal issues that web scraping would risk. Since the type of textual data we are searching for (the general opinion of LSE) is rarely provided in formatted tables online, using an the Reddit API is the best way to acquire and aggregate this information in a structured way.

Furthermore, the website has a strong user base for university students or people generally interested in the subject, exemplified by the size of the subreddit 'UniUK'. Not only does this give access to a large amount of data, but the posts are detailed and honest. This is because a significant part of the branding for Reddit is the idea of community, where you are likely to find other users who want contribute to the discussion under the protection of anonymity.

In [48]:
'''
Reddit API Data Collection.
Use the API key to request an access token by using the data stored in keys.json.
Make an API call to retrive posts from 'UniUK'
Store the ddata in a data frame.
'''

#access keys
with open('Data/keys.json') as f:
    keys = json.load(f)
app_id = keys['reddit']['app_id']
app_secret = keys['reddit']['app_secret']
username = keys['reddit']['username']
password = keys['reddit']['password']

#request a token
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
post_data = {'grant_type': 'password',
            'username': username,
            'password': password}
headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
r = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
access_token = r.json()['access_token']

#send a request
headers = {"Authorization": f"bearer {access_token}",
'User-Agent': f'lse/0.0.1 by {username}'}

r = requests.get("https://oauth.reddit.com/r/UniUK/search?q=lse" \
"&limit=30&sort=top&restrict_sr=true", headers=headers)

titles = []
scores = []
top_comments = []

#post title, score and top comment
for post in r.json()["data"]["children"]:
    post_title = post["data"]["title"]
    post_score = post["data"]["score"]
    post_permalink = post["data"]["permalink"]

    comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
    comments_data = comments_request.json()

    if isinstance(comments_data, list) and len(comments_data) > 1:
        comments = comments_data[1]["data"]["children"]
        if comments:
            top_comment = comments[0]["data"]["body"]
            titles.append(post_title)
            scores.append(post_score)
            top_comments.append(top_comment)

lse_reddit_df = pd.DataFrame({
    "Title": titles,
    "Score": scores,
    "Top Comment": top_comments
})

#save dataframe to csv file
lse_reddit_df.to_csv("Data/reddit_data.csv", index=False)


Reddit data collected on King's College London, University College London, and Imperial College London for further data analysis is collected below. When searching for data regarding these universities, the following abbreviations have been used for each university, as they gather more responses and are more commonly used to refer to the above universities online.

Kings, UCL, Imperial.

In [115]:
#access keys
with open('Data/keys.json') as f:
    keys = json.load(f)
app_id = keys['reddit']['app_id']
app_secret = keys['reddit']['app_secret']
username = keys['reddit']['username']
password = keys['reddit']['password']

#request a token
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
post_data = {'grant_type': 'password',
            'username': username,
            'password': password}
headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
r = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
access_token = r.json()['access_token']

#send a request
headers = {"Authorization": f"bearer {access_token}",
'User-Agent': f'lse/0.0.1 by {username}'}

r = requests.get("https://oauth.reddit.com/r/UniUK/search?q=ucl" \
"&limit=30&sort=top&restrict_sr=true", headers=headers)

titles = []
scores = []
top_comments = []

#post title, score and top comment
for post in r.json()["data"]["children"]:
    post_title = post["data"]["title"]
    post_score = post["data"]["score"]
    post_permalink = post["data"]["permalink"]

    comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
    comments_data = comments_request.json()

    if isinstance(comments_data, list) and len(comments_data) > 1:
        comments = comments_data[1]["data"]["children"]
        if comments:
            top_comment = comments[0]["data"]["body"]
            titles.append(post_title)
            scores.append(post_score)
            top_comments.append(top_comment)

ucl_reddit_df = pd.DataFrame({
    "Title": titles,
    "Score": scores,
    "Top Comment": top_comments
})

#save dataframe to csv file
ucl_reddit_df.to_csv("Data/ucl_reddit_data.csv", index=False)

In [ ]:
#access keys
with open('Data/keys.json') as f:
    keys = json.load(f)
app_id = keys['reddit']['app_id']
app_secret = keys['reddit']['app_secret']
username = keys['reddit']['username']
password = keys['reddit']['password']

#request a token
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
post_data = {'grant_type': 'password',
            'username': username,
            'password': password}
headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
r = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
access_token = r.json()['access_token']

#send a request
headers = {"Authorization": f"bearer {access_token}",
'User-Agent': f'lse/0.0.1 by {username}'}

r = requests.get("https://oauth.reddit.com/r/UniUK/search?q=kings" \
"&limit=30&sort=top&restrict_sr=true", headers=headers)

titles = []
scores = []
top_comments = []

#post title, score and top comment
for post in r.json()["data"]["children"]:
    post_title = post["data"]["title"]
    post_score = post["data"]["score"]
    post_permalink = post["data"]["permalink"]

    comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
    comments_data = comments_request.json()

    if isinstance(comments_data, list) and len(comments_data) > 1:
        comments = comments_data[1]["data"]["children"]
        if comments:
            top_comment = comments[0]["data"]["body"]
            titles.append(post_title)
            scores.append(post_score)
            top_comments.append(top_comment)

kings_reddit_df = pd.DataFrame({
    "Title": titles,
    "Score": scores,
    "Top Comment": top_comments
})

#save dataframe to csv file
kings_reddit_df.to_csv("Data/kings_reddit_data.csv", index=False)

In [ ]:
#access keys
with open('Data/keys.json') as f:
    keys = json.load(f)
app_id = keys['reddit']['app_id']
app_secret = keys['reddit']['app_secret']
username = keys['reddit']['username']
password = keys['reddit']['password']

#request a token
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
post_data = {'grant_type': 'password',
            'username': username,
            'password': password}
headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
r = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
access_token = r.json()['access_token']

#send a request
headers = {"Authorization": f"bearer {access_token}",
'User-Agent': f'lse/0.0.1 by {username}'}

r = requests.get("https://oauth.reddit.com/r/UniUK/search?q=imperial" \
"&limit=30&sort=top&restrict_sr=true", headers=headers)

titles = []
scores = []
top_comments = []

#post title, score and top comment
for post in r.json()["data"]["children"]:
    post_title = post["data"]["title"]
    post_score = post["data"]["score"]
    post_permalink = post["data"]["permalink"]

    comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
    comments_data = comments_request.json()

    if isinstance(comments_data, list) and len(comments_data) > 1:
        comments = comments_data[1]["data"]["children"]
        if comments:
            top_comment = comments[0]["data"]["body"]
            titles.append(post_title)
            scores.append(post_score)
            top_comments.append(top_comment)

imperial_reddit_df = pd.DataFrame({
    "Title": titles,
    "Score": scores,
    "Top Comment": top_comments
})

#save dataframe to csv file
imperial_reddit_df.to_csv("Data/imperial_reddit_data.csv", index=False)

To answer the question 'Has student satisfaction improved since COVID-19? (2020-2023)?', we then decided to search the same subreddit 'UniUK' but instead with the key work 'covid'*. Similar to previously, these posts and comments were stored in a csv titled 'covid_reddit_data.csv' in the 'Data' folder. We intended to search on Reddit specifially for LSE student opinions about COVID-19, however very few relevant posts were found. For example, most were referencing a research paper on COVID-19 published by an LSE staff member instead of student opinions on the pandemic. This could be noted as a positive sign that there was few complaints online that students had about LSE during lockdown.

We justified only searching the subreddit under the umbrella term 'covid' since we primarily wanted to see if there was an overall impact on student satisfaction for unversity students across the UK, and we could assume this also includes LSE. This also gave us a larger scope of data to collect.

*Note: the informal name 'covid' was used to denote the pandemic instead of 'COVID-19' as this generated more search results.

In [4]:
'''
Reddit API Data Collection.
Use the API key to request an access token by using the data stored in keys.json.
Make an API call to retrive posts from 'UniUK'
Store the ddata in a data frame.
'''

#access keys
with open('Data/keys.json') as f:
    keys = json.load(f)
app_id = keys['reddit']['app_id']
app_secret = keys['reddit']['app_secret']
username = keys['reddit']['username']
password = keys['reddit']['password']

#request a token
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
post_data = {'grant_type': 'password',
            'username': username,
            'password': password}
headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
r = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
access_token = r.json()['access_token']

#send a request
headers = {"Authorization": f"bearer {access_token}",
'User-Agent': f'lse/0.0.1 by {username}'}

r = requests.get("https://oauth.reddit.com/r/UniUK/search?q=covid" \
"&limit=200&sort=top&restrict_sr=true", headers=headers)

titles = []
scores = []
top_comments = []

#post title, score and top comment
for post in r.json()["data"]["children"]:
    post_title = post["data"]["title"]
    post_score = post["data"]["score"]
    post_permalink = post["data"]["permalink"]

    comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
    comments_data = comments_request.json()

    if isinstance(comments_data, list) and len(comments_data) > 1:
        comments = comments_data[1]["data"]["children"]
        if comments:
            top_comment = comments[0]["data"]["body"]
            titles.append(post_title)
            scores.append(post_score)
            top_comments.append(top_comment)

reddit_covid_df = pd.DataFrame({
    "Title": titles,
    "Score": scores,
    "Top Comment": top_comments
})

#save dataframe to csv file
reddit_covid_df.to_csv("Data/covid_reddit_data.csv", index=False)


In [131]:
#below is optimised code, BUT WILL MESS UP YOUR EXISITNG CODE SOPHIE for reddit stuff!! SO lets se

In [120]:
'''
Reddit API Data Collection.
Use the API key to request an access token by using the data stored in keys.json.
Make an API call to retrive posts from 'UniUK'
Store the data in a data frame.
'''

def fetch_reddit_data(query, limit):
    
    with open('Data/keys.json') as f:
        keys = json.load(f)
    app_id = keys['reddit']['app_id']
    app_secret = keys['reddit']['app_secret']
    username = keys['reddit']['username']
    password = keys['reddit']['password']
    
    # Request a token
    client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
    post_data = {'grant_type': 'password',
                'username': username,
                'password': password}
    headers = {'User-Agent': f'new connection lse/0.0.1 by {username}'}
    r = requests.post('https://www.reddit.com/api/v1/access_token',
                        auth=client_auth, data=post_data, headers=headers)
    access_token = r.json()['access_token']
    
    # Send a request
    headers = {"Authorization": f"bearer {access_token}",
               'User-Agent': f'lse/0.0.1 by {username}'}
    r = requests.get(f"https://oauth.reddit.com/r/UniUK/search?q={query}" \
                     f"&limit={limit}&sort=top&restrict_sr=true", headers=headers)
    
    titles = []
    scores = []
    top_comments = []
    
    # Post title, score, and top comment
    for post in r.json()["data"]["children"]:
        post_title = post["data"]["title"]
        post_score = post["data"]["score"]
        post_permalink = post["data"]["permalink"]

        comments_request = requests.get(f"https://oauth.reddit.com{post_permalink}.json", headers=headers)
        comments_data = comments_request.json()

        if isinstance(comments_data, list) and len(comments_data) > 1:
            comments = comments_data[1]["data"]["children"]
            if comments:
                top_comment = comments[0]["data"]["body"]
                titles.append(post_title)
                scores.append(post_score)
                top_comments.append(top_comment)
    
    reddit_df = pd.DataFrame({
        "Title": titles,
        "Score": scores,
        "Top Comment": top_comments
    })
    
    # Save dataframe to csv file
    reddit_df.to_csv(f"Data/{query}_reddit_data_c.csv", index=False)
    #ABOVE CODE NEEDS TO BE CHANGED, ITS CURRENTLY SAVING TO AN OFF FILE!!!!!

In [123]:
fetch_reddit_data('lse', 30)
fetch_reddit_data('kings', 30)
fetch_reddit_data('ucl', 30)
fetch_reddit_data('imperial', 30)
fetch_reddit_data('covid', 200)

In [130]:
#above code HAS NOT BEEN RAN YET, DO NOT RUN